In [2]:
import json

data = json.load(open('en_de_corpus.json', 'r'))
nSentences = 100

In [3]:
type(data)

dict

In [4]:
len(data)

2

In [5]:
len(data['en'])

132173

In [6]:
len(data['de'])

132173

In [7]:
data['en'][0]

"What do I have to do now that I'm registered?"

In [8]:
data['de'][0]

'Was mache ich jetzt, da ich angemeldet bin?'

In [9]:
#Script to remove punctuation, ensure <= max allowed length + padding

import string
import random
import math

#replace punctuation with space, ensure all sentences with  1 space
#also all capital to small 
u

#max allowed raw length=5, pad to make it 5, allow only equal pairs
tempEn = list(map(replacePunctuation,data['en']))
tempDe = list(map(replacePunctuation,data['de']))
maxRawLength = 5
maxFinalLength = maxRawLength + 2

def wordCount(x):
    return len(x.split())

data_en_processed = list()
data_de_processed = list()
for i in range(len(tempEn)):
    if(wordCount(tempEn[i]) <= maxRawLength and wordCount(tempDe[i]) <= maxRawLength
       and wordCount(tempEn[i])==wordCount(tempDe[i])):
        #if(i%10==0):
        #    print((maxFinalLength -1 - wordCount(tempEn[i])))
        data_en_processed.append('< '+tempEn[i]+' >'*(maxFinalLength -1 - wordCount(tempEn[i])))
        data_de_processed.append('< '+tempDe[i]+' >'*(maxFinalLength -1 - wordCount(tempDe[i])))

print("length after processing : "+str(len(data_en_processed)))  


length after processing : 21680


In [10]:
#pick 5 random samples and display, compare with google translate
import time
from mtranslate import translate


start = math.ceil(random.random()*1000)
for i in range(start,start+5):
    print(data_en_processed[i]+" | "+data_de_processed[i])
    print(translate(data_en_processed[i],'de','en'))
    print()

    

< where s the bag > > | < wo ist die tasche > >
<Wo s die Tasche>>

< radioactive matter is dangerous > > | < radioaktive stoffe sind gefährlich > >
<Radioaktive Stoffe ist gefährlich>>

< we have reached our destination > | < wir haben unser ziel erreicht >
<Wir haben unser Ziel erreicht>

< i wash my hands > > | < ich wasche meine hände > >
<Ich wasche meine Hände>>

< i think you are mistaken > | < ich denke du liegst verkehrt >
<Ich glaube du irrst dich



In [11]:
# Convert to list of lists from list of strings, remove space

data_en_processed_tokens = [s.split() for s in data_en_processed]
data_de_processed_tokens = [s.split() for s in data_de_processed]
print(data_de_processed_tokens[77])

['<', 'er', 'unterrichtet', 'arabisch', '>', '>', '>']


In [12]:
#Lower the size
data_en_processed_tokens = data_en_processed_tokens[:1000]
data_de_processed_tokens = data_de_processed_tokens[:1000]
data_full = data_en_processed_tokens + data_de_processed_tokens
len(data_full)

data_full = sum(data_full, [])
len(data_full)

14000

In [13]:
print(data_full[0:7])
print(data_full[0+7000:7000+7])

['<', 'i', 'took', 'the', 'bus', 'back', '>']
['<', 'ich', 'nahm', 'den', 'bus', 'zurück', '>']


In [15]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np
from mtranslate import translate


# data I/O
#data = open('input_en_de_1000.txt', 'r').read() # should be simple plain text file
data = data_full
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters---> 43     loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

hidden_size = 150 # size of hidden layer of neurons
seq_length = 7 # number of steps to unroll the RNN for
learning_rate = 1e-2 #1e-1

#globals
en_ch_cnt = 7000 #len(data_full)//2
test_sentence = ['<', 'i', 'took', 'the', 'bus', 'back', '>']
#'<', 'ich', 'nahm', 'den', 'bus', 'zurück', '>'

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
   # print("t",t)
   # print("shape",(ps[t]).shape)
   # print("targets",targets[t])
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """

  n = len(test_sentence)

  


  ixes = []
  for t in range(n):
    x = np.zeros((vocab_size, 1))
    x[char_to_ix[test_sentence[t]]] = 1
        
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    probab = np.exp(y) / np.sum(np.exp(y))
    #ix = np.argmax(probab)
    ix = np.random.choice(range(vocab_size), p=probab.ravel())
    
    
    
    #x = np.zeros((vocab_size, 1))
    #x[ix] = 1
    
    
    
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data[0:en_ch_cnt]) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  #targets = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+en_ch_cnt:p+seq_length+en_ch_cnt]]

  # sample from the model now and then
  if n % 1000 == 0:
    for i in range(4):
        slen = len(test_sentence)
        sample_ix = sample(hprev, inputs[0], slen)
        txt = ' '.join(ix_to_char[ix] for ix in sample_ix)
        #print('----\n %s \n----' % (txt, ))
        print('%s \n----' % (txt, ))
        #print(translate(txt,'en', 'de'))
        #print('---------------')
    print('-------------------------------------------------')
    


  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 14000 characters, 2651 unique.
thought sugar bewundern wish wollte ampel führt 
----
wealthy vergöttert selfish day river bin missed 
----
leave sterben wait obstinate jemals pferdestärken bruder 
----
aßen roman master advice weiter always wen 
----
-------------------------------------------------
iter 0, loss: 55.178845
< er maria ist stiefschwestern alt > 
----
< sie sieben esse öle hobby > 
----
< ich sun pronunciation mouth luft > 
----
< haben tom höre ab artikel > 
----
-------------------------------------------------
iter 1000, loss: 38.654577
< ich nett niedrig kontakt verfügung > 
----
< papier hat macht reagier verkehrt > 
----
< ich schläger ist betrunken kommen > 
----
< ich czech ihre zweierreihen reparieren > 
----
-------------------------------------------------
iter 2000, loss: 29.762010
< ich coming annehmbar allein ausgleichssport > 
----
< wir verschwunden sehr jung uhr > 
----
< mir meinung wird thema talkative > 
----
< ironed meiner schmidt vernachläs

KeyboardInterrupt: 

In [10]:
data_en_processed

['< i took the bus back >',
 '< without air we would die >',
 '< i study chinese in beijing >',
 '< pass me the salt please >',
 '< the boy is wearing glasses >',
 '< we know this song > >',
 '< that was hard to believe >',
 '< he has blond hair > >',
 '< thanks for your patience > >',
 '< tom goes to bed early >',
 '< lions are stronger than wolves >',
 '< this flower smells sweet > >',
 '< this book makes pleasant reading >',
 '< tom swims very fast > >',
 '< we ll hurry > > >',
 '< i love the sun > >',
 '< can this be it > >',
 '< she s afraid to sing >',
 '< this is useless > > >',
 '< i ll call her immediately >',
 '< my grandfather gets up early >',
 '< you sell us fish > >',
 '< birds often fly together > >',
 '< i love serious music > >',
 '< is tom there alone > >',
 '< we are the borg > >',
 '< how much is this radio >',
 '< they marched two abreast > >',
 '< i like traveling > > >',
 '< do what you want > >',
 '< she has tiny feet > >',
 '< give it to me please >',
 '< can y

In [18]:
data_de_processed[1]

'< ohne luft würden wir sterben >'

In [36]:
count = 0
for i in data.keys():
    for sentence in data[i]:
        for char in sentence:
            count += 1
        
print(count//2)

5050039


In [59]:
list(data.keys())

['en', 'de']

In [60]:
cnt = 0
for sentence in data['en']:
    for char in sentence:
        cnt += 1
        
print(cnt)

4669629


In [61]:
cnt = 0
for sentence in data['de']:
    for char in sentence:
        cnt += 1
        
print(cnt)

5430450


In [62]:
cnt = 0
for sentence in data['de']:
        cnt += len(sentence.split(' '))
        
print(cnt)
print(cnt/132173)

913058
6.908052325361458


In [63]:
print(char)

.


In [64]:
src_txts = data['en']
tgt_txts = data['de']

MAX_LEN = 7

en_txts, de_txts = [], []
for i, src in enumerate(src_txts):
    if len(src.split(' ')) <= MAX_LEN:
        tgt = tgt_txts[i]
        if len(tgt.split(' ')) <= MAX_LEN:
            en_txts.append(src)
            de_txts.append(tgt)
            


In [65]:
len(en_txts)

79512

In [66]:
len(de_txts)

79512

In [67]:
en_txts[0], de_txts[0]

('I took the bus back.', 'Ich nahm den Bus zurück.')

In [68]:
en_txts[-1], de_txts[-1]

('I am the fastest runner.', 'Ich bin der schnellste Läufer.')

In [69]:
cnt = 0
for sentence in data['de']:
    if not set(sentence).isdisjoint(set("<>")) :
        cnt += 1
        
print(cnt)

0


We decrease size of dataset temporarily

In [70]:
en_txts = en_txts[ :nSentences]
de_txts = de_txts[ :nSentences]
print( len(en_txts), len(de_txts))

100 100


In [71]:
en_txts = en_txts[ :nSentences]
print("".join(en_txts)) #282 chars #28357


I took the bus back.I'm neither an Athenian nor a Greek.He had barely enough to eat.We are traveling to Barcelona.Without air we would die.I thought you'd be older.I thought you wouldn't like me.Hanako questioned his sincerity.I study Chinese in Beijing.I assume it was unavoidable.Pass me the salt, please.I guess the battery is flat.Tom is rarely late.Am I wrong?He admitted he had taken bribes.No one knew who owned the land.The boy is wearing glasses.My sister was whistling merrily.Bahrain became a kingdom in 2002.You know it better than me.What are your plans for Christmas?We know this song.That was hard to believe.I don't believe a word you say.Our money ran out.Thank you for clearing up the misunderstanding.The town lies on the seashore.He has blond hair.The magazine is aimed at teenagers.Thanks for your patience.Tom likes to do things by himself.Tom goes to bed early.We walked to the river.You've got the wrong number.Lions are stronger than wolves.This flower smells sweet.This book

In [72]:
set("".join(en_txts))

{' ',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '5',
 '6',
 '9',
 '?',
 'A',
 'B',
 'C',
 'D',
 'F',
 'G',
 'H',
 'I',
 'L',
 'M',
 'N',
 'O',
 'P',
 'S',
 'T',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z'}

In [73]:
de_txts = de_txts[ :1000]
print("".join(de_txts)) #330 chars #32600

Ich nahm den Bus zurück.Ich bin weder Athener, noch Grieche.Er hatte kaum genug zu essen.Wir sind auf der Reise nach Barcelona.Ohne Luft würden wir sterben.Ich hätte Sie für älter gehalten.Ich dachte, du würdest mich nicht mögen.Hanako zweifelte an seiner Ernsthaftigkeit.Ich lerne in Peking Chinesisch.Das war wohl unvermeidlich.Reich mir das Salz, bitte.Ich schätze, die Batterie ist leer.Tom kommt selten zu spät.Hab ich nicht Recht?Er gab zu, bestochen worden zu sein.Niemand wusste, wem das Land gehörte.Der Junge trägt eine Brille.Meine Schwester pfiff munter vor sich hin.Bahrain wurde 2002 ein Königreich.Du weißt das besser als ich.Was hast du zu Weihnachten vor?Wir kennen dieses Lied.Das war schwer zu glauben.Ich glaube dir kein Wort.Das Geld ging uns aus.Danke, dass du das Missverständnis geklärt hast.Die Stadt liegt an der Meeresküste.Er hat blonde Haare.Die Zeitschrift richtet sich an Teenager.Danke für Ihre Geduld!Tom arbeitet gern allein.Tom geht früh ins Bett.Wir liefen zum Flu

In [74]:
60957 - 28357 

32600

In [75]:
en_txts_ss = [' '.join(["<", sentence, ">"]) for sentence in en_txts]
de_txts_ss = [' '.join(["<", sentence, ">"]) for sentence in de_txts]

In [76]:
en_txts_ss[0]

'< I took the bus back. >'

In [77]:
de_txts_ss[0]

'< Ich nahm den Bus zurück. >'

In [78]:
#How many unique characters
len(list(set("".join(en_txts) ))) , len(list(set("".join(de_txts) )))

(54, 61)

In [79]:
uniq_en = sorted(list(set("".join(en_txts) )))
uniq_de = sorted(list(set("".join(de_txts) )))
ENG_CHARS = "".join(uniq_en)
DE_CHARS ="".join(uniq_de)

In [80]:
ENG_CHARS

" ',-.012569?ABCDFGHILMNOPSTWYabcdefghijklmnopqrstuvwyz"

In [81]:
DE_CHARS

' !,.012569?ABCDEFGHIJKLMNOPRSTVWZabcdefghijklmnoprstuvwyzßäöü'

In [ ]:
#Copy paste both lines in both corpora

In [ ]:
from IPython.core.debugger import Tracer; Tracer()() 

/home/shinchan/MLProject/venv/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  if __name__ == '__main__':


--Call--
> /home/shinchan/MLProject/venv/local/lib/python3.5/site-packages/IPython/core/displayhook.py(236)__call__()
    234         sys.stdout.flush()
    235 
--> 236     def __call__(self, result=None):
    237         """Printing with history cache management.
    238 

ipdb> 
ipdb> 
ipdb> 
ipdb> 
--KeyboardInterrupt--


In [ ]:
%whos

In [ ]:
import string
puncAndNum = string.punctuation+'01233456789'

#replace punctuation with space
def replacePunctuation(x):
    x = x.lower()
    for char in x:
        if char in puncAndNum:
            #print(char)
            x=x.replace(char,' ')
    return ' '.join(x.split())


In [ ]:
replacePunctuation("hello 93 I'm 45 ? dwsdsa ddsa. ds")

In [ ]:
enNoPunctuation = (replacePunctuation("".join(en_txts)))
print(enNoPunctuation)
len(enNoPunctuation)

In [ ]:
deNoPunctuation = (replacePunctuation("".join(de_txts)))
print(deNoPunctuation)

In [ ]:
combined = open("input_en_de_nopunctuation_100.txt", "r").read()
wordList = combined.split(" ")
#print(wordList)
#print(combined)
wordCount = 0
wordToIndexMap = {}
indexToWordMap = {}
for word in wordList:
    #print(word)
    if(not(word in wordToIndexMap)):
        wordCount += 1
        wordToIndexMap[word] = wordCount
        indexToWordMap[wordCount] = word



In [ ]:
#wordToIndexMap
#len(wordMap)

def getIndex(x):
    return wordToIndexMap[x]


In [ ]:
wordListIndexes = list(map(getIndex,wordList))

In [ ]:
#print(wordListIndexes)
for word in wordList:
    if(word!=indexToWordMap[wordToIndexMap[word]]):
        print("fail")

In [ ]:
print(wordList)

In [ ]:
# data I/O
data =  wordList #open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [ ]:
ix_to_char

In [ ]:
char_to_ix